# Grove Analog ports and PWM

The arduino shield has dedicated analog ports which can be accessed through the `Analog` peripheral. This peripheral can only be used with the A1 to A4 ports on the PYNQ shield and not at all using the PMOD interface. First we'll load the library and initialise the base overlay.

In [1]:
import ipython_microblaze as ipmb

from pynq.overlays.base import BaseOverlay
base = BaseOverlay('base.bit')

Following the other APIs the Analog API consists of the following functions

```C
mb_adc iop_switch_analog_grove(unsigned char port, unsigned char wire);
mb_adc iop_switch_analog_raw(unsigned char pin);

mb_adc mb_adc_open(int channel);
int mb_adc_read_raw(mb_adc);
float mb_adc_read(mb_adc);

int mb_adc_range(mb_adc);
float mb_adc_vref(mb_adc);
```

This peripheral follows the format of many in that upon connecting the pin, a handle is returned which can be used for reading the values back. `mb_adc_read_raw` returns a number between 0 and MAX where MAX can be found using `mb_adc_range`. `mb_adc_read` will do the conversion to the voltage for you.

For a simple example, let's create a program which reads the value from the A1 port and sends it back to python.

In [2]:
%%microblaze base.ARDUINO analog_test ipmb.Analog
#include <unistd.h>
#include <iop_switch.h>

int main() {
    iop_switch_init();
    mb_adc conn = iop_switch_analog_grove(A1, 0);
    int val = mb_adc_read_raw(conn);
    write(STDOUT_FILENO, &val, 4);
}

In [3]:
import struct
print(struct.unpack('I', analog_test.stream.read()))

analog_test.reset()

(6387,)


There is an additional function `adc_connect_i2c(i2c bus, unsigned int address)` which allows for I2C attached ADCs to be used with the same API. This is shown in the Microblaze Compilation notebook to read from a sensor attached to a PMOD interface.

Next we can introduce the PWM API which generates a regular stream of pulses on an output pin.

```C
mb_timer iop_switch_pwm_raw(unsigned char pin);
mb_timer iop_switch_pwm_grove(unsigned char port, unsigned char wire);

mb_timer mb_timer_open(int index);
void mb_timer_delay_us(mb_timer timer, int channel, int usdelay);
void mb_timer_pwm_generate(mb_timer timer, int pulse, int period);
void mb_timer_pwm_stop(mb_timer timer);
```

This peripheral driver uses the same idiom as the `Analog` driver. The PWM has some limitations to be aware of - for the Arduino output it can only be connected to digital pins D3, D5, D6, D9, D10 and D11 however all of these outputs can be used in parallel. For the PYNQ shield this restricts single-wire Grove peripherals to the G2, G4 and G6 connectors.

The `mb_timer_pwm_generate` function starts the PWM generator and set the period and pulse width measured in cycles. `mb_timer_pwm_stop` drives the output at 0.

For an example we can connect a sensor to A1 and an output such as an LED to G2 and use that to represent the current value of the input.

In [4]:
%%microblaze base.ARDUINO pwm_test
#include <iop_switch.h>

int main() {
    iop_switch_init();
    mb_adc input = iop_switch_analog_grove(A1, 0);
    mb_timer output = iop_switch_pwm_grove(G2, 0);
    int range = mb_adc_range(input);
    int val = 0;
    while (1) {
        val = mb_adc_read_raw(input);
        mb_timer_pwm_generate(output, val, range);
    }
}

In [5]:
pwm_test.reset()

The PWM library can also be used on a PMOD output although here there is only a single generator but it can be routed to any of the output pins. This example ramps an LED attached to G2

In [6]:
%%microblaze base.PMODA pwm_test
#include <iop_switch.h>

int main() {
    iop_switch_init();
    mb_timer output = iop_switch_pwm_grove(G2, 0);
    while (1) {
        for (int i = 0; i < 100; ++i) {
            mb_timer_pwm_generate(output, i, 100);
            for (int j = 0; j < 1000000; ++j);
        }
    }
}

In [7]:
pwm_test.reset()